In [24]:
import json
from pathlib import Path

from openai import OpenAI
from pydantic import BaseModel

In [2]:
llm = OpenAI()

In [3]:
files = list(Path('../data').rglob('*.md'))

In [4]:
test = files[0]

In [5]:
with open(test, "r") as f:
    email = f.read()

In [35]:
with open("../data/reading_history.txt", "r") as f:
    reading_history = f.read()

with open("prompts/extraction.txt", "r") as f:
    prompt_extraction = f.read()

with open("prompts/infer_interests.txt", "r") as f:
    prompt_interest = f.read()

In [29]:
class NewsletterItem(BaseModel):
    headline: str
    url: str
    description: str

class Newsletter(BaseModel):
    title: str
    items: list[NewsletterItem]

In [30]:
resp = llm.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": prompt_extraction
        },
        {
            "role": "user",
            "content": email
        }
    ],
    response_format=Newsletter
)

In [31]:
json_string = resp.choices[0].message.content

In [32]:
# parse the json string into a python object
parsed = json.loads(json_string)

In [43]:
resp = llm.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": prompt_interest
        },
        {
            "role": "user",
            "content": reading_history
        }
    ]
)

In [44]:
print(resp.choices[0].message.content)

Based on the diverse range of articles read by this user, it appears that they have a keen interest in several key areas:

1. **Politics and Elections**: Many articles focus on political figures, election dynamics, and polling, especially related to Kamala Harris, Donald Trump, and the upcoming Senate races. This indicates a strong interest in U.S. politics, election strategies, and the implications of various political events.

2. **Economic and Labor Issues**: The user reads articles addressing employment statistics and economic challenges, suggesting an awareness of the labor market and its influences on wider economic trends.

3. **Technology and AI**: Several articles delve into topics related to artificial intelligence, its implications for different sectors (including media and workplaces), and controversies surrounding tech giants. This suggests a deep interest in the intersection of technology with everyday life and its societal impacts.

4. **Social Issues and Safety**: Artic